# BJT as Switch and Amplifier
## 2N3904/2N3906 Experiments — Switching, Biasing, and Common-Emitter Gain

**Module 03 — Transistors (BJT) | Notebook 01**

---

Now that we understand how BJTs work internally, we put them to work in the two most
fundamental configurations: as a **switch** (digital) and as an **amplifier** (analog).

**What you will learn:**
- How to design a reliable BJT switch circuit
- Choosing base resistor values and ensuring saturation
- The common-emitter amplifier: biasing and voltage gain
- PNP transistors for high-side switching
- Power dissipation considerations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import ipywidgets as widgets
from IPython.display import display, HTML

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

---
## Concept — BJT as a Switch

When used as a switch, the BJT operates in only two states:

- **OFF (Cutoff)**: No base current, no collector current. V_CE = V_supply.
- **ON (Saturation)**: Plenty of base current, maximum collector current. V_CE ~ V_CE(sat) ~ 0.2V.

The BJT switch is the foundation of digital electronics. Every logic gate in the earliest
computers was built from BJT switches.

### Basic NPN Switch Circuit

```
    V_CC (+5V to +12V)
         |
        [R_C]  (Load — LED, relay, motor, etc.)
         |
         C
    B ---| 2N3904 (NPN)
    |     E
   [R_B]  |
    |    GND
  V_in
  (logic signal)
```

### Design Rules for Reliable Switching

1. **Calculate the required collector current**: I_C = (V_CC - V_load) / R_C
2. **Calculate minimum base current**: I_B(min) = I_C / beta(min)
3. **Overdrive the base** — use a "forced beta" of ~10 for reliable saturation:
   I_B(design) = I_C / 10
4. **Calculate base resistor**: R_B = (V_in - V_BE) / I_B(design)

**Why overdrive?** Beta varies from part to part (100-300 for 2N3904). If you design for
beta = 200 and get a transistor with beta = 100, your switch won't saturate properly.
Using a forced beta of 10 guarantees saturation regardless of the actual beta.

In [ ]:
# Calculate base resistor for switching an LED

def design_bjt_switch(V_CC, V_in, V_LED, I_LED_mA, V_BE=0.7, V_CE_sat=0.2, forced_beta=10):
    """Design a BJT switch circuit for driving an LED."""
    I_LED = I_LED_mA / 1000  # convert to amps
    
    # Step 1: Calculate R_C (current-limiting resistor for LED)
    V_RC = V_CC - V_LED - V_CE_sat
    R_C = V_RC / I_LED
    
    # Step 2: I_C = I_LED (through the LED and R_C)
    I_C = I_LED
    
    # Step 3: Required base current with forced beta
    I_B = I_C / forced_beta
    
    # Step 4: Base resistor
    R_B = (V_in - V_BE) / I_B
    
    # Standard resistor values (E24 series - a few common ones)
    standard_values = [100, 120, 150, 180, 220, 270, 330, 390, 470, 560, 680, 820,
                       1000, 1200, 1500, 1800, 2200, 2700, 3300, 3900, 4700, 5600, 
                       6800, 8200, 10000, 12000, 15000, 18000, 22000, 27000, 33000,
                       39000, 47000, 56000, 68000, 82000, 100000]
    
    # Find nearest standard values
    R_C_std = min(standard_values, key=lambda x: abs(x - R_C))
    R_B_std = min(standard_values, key=lambda x: abs(x - R_B))
    
    print("=" * 55)
    print("BJT SWITCH DESIGN — LED Driver")
    print("=" * 55)
    print(f"\nGiven:")
    print(f"  V_CC (supply)      = {V_CC} V")
    print(f"  V_in (logic input) = {V_in} V")
    print(f"  LED forward voltage = {V_LED} V")
    print(f"  Desired LED current = {I_LED_mA} mA")
    print(f"  V_BE (assumed)     = {V_BE} V")
    print(f"  V_CE(sat) (assumed)= {V_CE_sat} V")
    print(f"  Forced beta        = {forced_beta}")
    
    print(f"\nCalculations:")
    print(f"  R_C = (V_CC - V_LED - V_CE(sat)) / I_LED")
    print(f"       = ({V_CC} - {V_LED} - {V_CE_sat}) / {I_LED*1000} mA")
    print(f"       = {R_C:.0f} ohms  ->  Use {R_C_std} ohms (standard)")
    print(f"")
    print(f"  I_C  = {I_C * 1e3:.1f} mA")
    print(f"  I_B  = I_C / forced_beta = {I_C*1e3:.1f} / {forced_beta} = {I_B*1e6:.0f} uA")
    print(f"  R_B  = (V_in - V_BE) / I_B")
    print(f"       = ({V_in} - {V_BE}) / {I_B*1e6:.0f} uA")
    print(f"       = {R_B:.0f} ohms  ->  Use {R_B_std} ohms (standard)")
    
    # Verify with actual beta range
    I_B_actual = (V_in - V_BE) / R_B_std
    print(f"\nVerification with standard resistor values:")
    print(f"  Actual I_B with R_B = {R_B_std} ohms: {I_B_actual*1e6:.0f} uA")
    print(f"  For beta = 100: I_C(max) = {100 * I_B_actual * 1e3:.1f} mA (saturated, I_C limited by R_C)")
    print(f"  For beta = 300: I_C(max) = {300 * I_B_actual * 1e3:.1f} mA (still saturated)")
    print(f"  Actual I_C (saturated) = (V_CC - V_LED - V_CE(sat)) / R_C = {(V_CC - V_LED - V_CE_sat) / R_C_std * 1e3:.1f} mA")
    
    # Power dissipation in the BJT
    I_C_sat = (V_CC - V_LED - V_CE_sat) / R_C_std
    P_BJT = V_CE_sat * I_C_sat + V_BE * I_B_actual
    print(f"\n  Power in BJT: P = V_CE(sat)*I_C + V_BE*I_B = {P_BJT*1e3:.1f} mW")
    print(f"  (Well within the 2N3904's 625 mW limit)")
    
    return R_C_std, R_B_std

# Design: 5V supply, 5V logic input, red LED (2.0V), 15mA target current
R_C, R_B = design_bjt_switch(V_CC=5.0, V_in=5.0, V_LED=2.0, I_LED_mA=15)

In [ ]:
# Plot load line on IC-VCE curves showing operating point

fig, ax = plt.subplots(figsize=(12, 7))

V_CC = 5.0
R_C = 180  # ohms (LED + resistor total effective resistance)
beta = 200
V_A = 100

V_CE_range = np.linspace(0, V_CC + 0.5, 500)

# Plot IC-VCE curves
I_B_values = [0, 50e-6, 100e-6, 150e-6, 200e-6, 300e-6, 500e-6]
colors = plt.cm.Blues(np.linspace(0.3, 1.0, len(I_B_values)))

for I_B_val, color in zip(I_B_values, colors):
    I_C = np.zeros_like(V_CE_range)
    for i, vce in enumerate(V_CE_range):
        if I_B_val == 0:
            I_C[i] = 0
        elif vce < 0.2:
            I_C[i] = beta * I_B_val * (vce / 0.2)
        else:
            I_C[i] = beta * I_B_val * (1 + vce / V_A)
    I_C = np.clip(I_C, 0, 0.05)  # clip for display
    label = f'I_B = {I_B_val*1e6:.0f} uA' if I_B_val > 0 else 'I_B = 0 (cutoff)'
    ax.plot(V_CE_range, I_C * 1e3, color=color, linewidth=1.5, label=label)

# Load line: I_C = (V_CC - V_CE) / R_total
# For LED circuit, effective R_total = R_C (simplified)
R_total = 180
I_C_loadline = np.maximum((V_CC - V_CE_range) / R_total, 0)
ax.plot(V_CE_range, I_C_loadline * 1e3, 'r-', linewidth=3, label='Load line', zorder=5)

# Mark the operating points
# Cutoff point (I_B = 0)
ax.plot(V_CC, 0, 'rs', markersize=15, zorder=6, markeredgecolor='black', markeredgewidth=2)
ax.annotate('OFF\n(Cutoff)', xy=(V_CC, 0), xytext=(V_CC - 0.5, 5),
            fontsize=11, fontweight='bold', color='darkred',
            arrowprops=dict(arrowstyle='->', color='darkred', lw=2),
            bbox=dict(boxstyle='round', facecolor='lightyellow'))

# Saturation point (high I_B)
V_CE_sat = 0.2
I_C_sat = (V_CC - V_CE_sat) / R_total
ax.plot(V_CE_sat, I_C_sat * 1e3, 'go', markersize=15, zorder=6, 
        markeredgecolor='black', markeredgewidth=2)
ax.annotate('ON\n(Saturation)', xy=(V_CE_sat, I_C_sat * 1e3), xytext=(1.5, I_C_sat * 1e3 + 3),
            fontsize=11, fontweight='bold', color='darkgreen',
            arrowprops=dict(arrowstyle='->', color='darkgreen', lw=2),
            bbox=dict(boxstyle='round', facecolor='lightyellow'))

# Active region Q-point example
I_B_q = 50e-6
I_C_q = beta * I_B_q
V_CE_q = V_CC - I_C_q * R_total
if V_CE_q > 0:
    ax.plot(V_CE_q, I_C_q * 1e3, 'mo', markersize=15, zorder=6,
            markeredgecolor='black', markeredgewidth=2)
    ax.annotate('Q-point\n(Active/Linear)', xy=(V_CE_q, I_C_q * 1e3), 
                xytext=(V_CE_q + 0.8, I_C_q * 1e3 - 5),
                fontsize=11, fontweight='bold', color='purple',
                arrowprops=dict(arrowstyle='->', color='purple', lw=2),
                bbox=dict(boxstyle='round', facecolor='lightyellow'))

ax.set_xlabel('V_CE (V)', fontsize=13)
ax.set_ylabel('I_C (mA)', fontsize=13)
ax.set_title('Load Line Analysis — Three Operating Points\n'
             '(Switch uses Cutoff <-> Saturation; Amplifier uses Active region)',
             fontsize=13, fontweight='bold')
ax.legend(loc='upper right', fontsize=9)
ax.set_xlim(-0.2, V_CC + 1)
ax.set_ylim(-1, 30)

plt.tight_layout()
plt.show()

---
## Concept — BJT as a Common-Emitter Amplifier

When biased in the **active region**, the BJT amplifies signals. The most common
configuration is the **Common-Emitter (CE)** amplifier:

```
    V_CC (+9V)
      |
     [R_C = 2.2k]        Output taken here (V_out)
      |                        |
      +------------------------+
      |
      C
  B--| 2N3904 (NPN)
  |   E
  |   |
  |  [R_E = 220]    (emitter degeneration — stabilizes bias)
  |   |
  |  GND
  |
  +---[R2 = 10k]--- GND
  |
  +---[R1 = 33k]--- V_CC
  |
  +---||--- V_in (AC coupled through 10uF capacitor)
```

### Key Design Points

- **Voltage divider bias** (R1, R2) sets the DC operating point independent of beta
- **R_E** provides negative feedback — stabilizes the bias point
- **Voltage gain** (simplified): A_v = -R_C / r_e, where r_e = 26mV / I_C
- The negative sign means the output is **inverted** (180 degrees phase shift)
- With R_E (no bypass cap): A_v ~ -R_C / R_E (more stable but lower gain)

In [ ]:
# Calculate voltage gain of common-emitter amplifier

def design_ce_amplifier(V_CC, R_C, R_E, R1, R2, beta=200):
    """Design and analyze a common-emitter amplifier."""
    print("=" * 55)
    print("COMMON-EMITTER AMPLIFIER ANALYSIS")
    print("=" * 55)
    
    # DC Bias Analysis
    V_BB = V_CC * R2 / (R1 + R2)  # Thevenin voltage at base
    R_BB = R1 * R2 / (R1 + R2)     # Thevenin resistance
    V_BE = 0.7
    
    # KVL: V_BB = V_BE + I_E * R_E (assuming I_E ~ I_C for large beta)
    I_E = (V_BB - V_BE) / R_E
    I_C = I_E * beta / (beta + 1)  # more precise
    I_B = I_C / beta
    V_CE = V_CC - I_C * R_C - I_E * R_E
    
    print(f"\nDC Bias Point:")
    print(f"  V_BB (base divider voltage) = {V_CC} * {R2}/({R1}+{R2}) = {V_BB:.2f} V")
    print(f"  I_E = (V_BB - V_BE) / R_E = ({V_BB:.2f} - {V_BE}) / {R_E} = {I_E*1e3:.2f} mA")
    print(f"  I_C ~ {I_C*1e3:.2f} mA")
    print(f"  I_B = {I_B*1e6:.1f} uA")
    print(f"  V_CE = {V_CC} - {I_C*1e3:.2f}mA*{R_C} - {I_E*1e3:.2f}mA*{R_E} = {V_CE:.2f} V")
    
    if V_CE < 0.3:
        print(f"  WARNING: V_CE = {V_CE:.2f}V — transistor may be saturated!")
    elif V_CE > V_CC - 0.5:
        print(f"  WARNING: V_CE = {V_CE:.2f}V — transistor may be near cutoff!")
    else:
        print(f"  V_CE = {V_CE:.2f}V — good, transistor is in active region.")
    
    # AC Analysis
    r_e = 0.026 / I_C  # thermal voltage / I_C (in ohms)
    
    # Gain without bypass capacitor on R_E
    A_v_with_RE = -R_C / (r_e + R_E)
    
    # Gain with bypass capacitor (AC shorts R_E)
    A_v_bypassed = -R_C / r_e
    
    print(f"\nAC Small-Signal Analysis:")
    print(f"  r_e = 26mV / I_C = 26mV / {I_C*1e3:.2f}mA = {r_e:.1f} ohms")
    print(f"")
    print(f"  Without bypass cap: A_v = -R_C / (r_e + R_E) = -{R_C}/({r_e:.1f}+{R_E})")
    print(f"                      A_v = {A_v_with_RE:.1f} ({20*np.log10(abs(A_v_with_RE)):.1f} dB)")
    print(f"")
    print(f"  With bypass cap:    A_v = -R_C / r_e = -{R_C}/{r_e:.1f}")
    print(f"                      A_v = {A_v_bypassed:.0f} ({20*np.log10(abs(A_v_bypassed)):.1f} dB)")
    print(f"")
    print(f"  (Negative gain = output is inverted 180 degrees)")
    
    # Maximum output swing
    V_swing_pos = V_CC - I_C * R_C - V_CE  # headroom to cutoff
    V_swing_neg = V_CE - 0.2  # headroom to saturation
    V_swing = min(V_swing_pos, V_swing_neg)
    print(f"\nMax output swing: +/- {V_swing:.2f} V (peak) around V_CE = {V_CE:.2f} V")
    
    return I_C, V_CE, A_v_with_RE, A_v_bypassed

# Design example
I_C, V_CE, Av1, Av2 = design_ce_amplifier(
    V_CC=9.0, R_C=2200, R_E=220, R1=33000, R2=10000, beta=200
)

In [ ]:
# Interactive widget: adjust R_C, R_B and see operating point move on load line

def interactive_load_line(R_C=1000, R1_k=33, R2_k=10, R_E=220, V_CC=9.0):
    """Interactive load line analysis for CE amplifier."""
    R1 = R1_k * 1000
    R2 = R2_k * 1000
    beta = 200
    V_BE = 0.7
    V_A = 100
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # ---- Left: Load line on IC-VCE curves ----
    V_CE_range = np.linspace(0, V_CC + 0.5, 500)
    
    # Plot IC-VCE curves
    for ib_ua in [0, 5, 10, 15, 20, 30, 40, 50]:
        ib = ib_ua * 1e-6
        ic = np.zeros_like(V_CE_range)
        for i, vce in enumerate(V_CE_range):
            if ib == 0:
                ic[i] = 0
            elif vce < 0.2:
                ic[i] = beta * ib * (vce / 0.2)
            else:
                ic[i] = beta * ib * (1 + vce / V_A)
        ax1.plot(V_CE_range, ic * 1e3, 'gray', linewidth=0.7, alpha=0.5)
    
    # Load line: V_CE = V_CC - I_C*(R_C + R_E)
    R_total = R_C + R_E
    I_C_load = np.maximum((V_CC - V_CE_range) / R_total, 0)
    ax1.plot(V_CE_range, I_C_load * 1e3, 'r-', linewidth=2.5, label='Load line')
    
    # Q-point
    V_BB = V_CC * R2 / (R1 + R2)
    I_E_q = max((V_BB - V_BE) / R_E, 0) if R_E > 0 else (V_BB - V_BE) / 100
    I_C_q = I_E_q * beta / (beta + 1)
    V_CE_q = V_CC - I_C_q * R_C - I_E_q * R_E
    V_CE_q = max(V_CE_q, 0.1)
    
    color = 'green' if 0.5 < V_CE_q < V_CC - 0.5 else 'red'
    ax1.plot(V_CE_q, I_C_q * 1e3, 'o', color=color, markersize=14, zorder=5,
             markeredgecolor='black', markeredgewidth=2, label='Q-point')
    
    ax1.set_xlabel('V_CE (V)', fontsize=12)
    ax1.set_ylabel('I_C (mA)', fontsize=12)
    ax1.set_title('Load Line & Q-Point', fontsize=13, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.set_xlim(0, V_CC + 0.5)
    ax1.set_ylim(0, V_CC / R_total * 1e3 * 1.2)
    
    # ---- Right: Info panel ----
    ax2.axis('off')
    r_e = 0.026 / max(I_C_q, 1e-6)
    A_v = -R_C / (r_e + R_E) if (r_e + R_E) > 0 else 0
    A_v_bypass = -R_C / r_e if r_e > 0 else 0
    P_diss = V_CE_q * I_C_q
    
    status = 'ACTIVE (good)' if 0.5 < V_CE_q < V_CC - 0.5 else 'OUT OF RANGE'
    
    info_text = (
        f"DC BIAS ANALYSIS\n"
        f"{'='*35}\n"
        f"V_BB = {V_BB:.2f} V\n"
        f"I_C  = {I_C_q*1e3:.2f} mA\n"
        f"V_CE = {V_CE_q:.2f} V\n"
        f"P_diss = {P_diss*1e3:.1f} mW\n"
        f"Status: {status}\n"
        f"\nAC GAIN ANALYSIS\n"
        f"{'='*35}\n"
        f"r_e = {r_e:.1f} ohms\n"
        f"A_v (no bypass) = {A_v:.1f}\n"
        f"A_v (bypassed)  = {A_v_bypass:.0f}\n"
        f"\nCOMPONENT VALUES\n"
        f"{'='*35}\n"
        f"R_C = {R_C} ohms\n"
        f"R_E = {R_E} ohms\n"
        f"R1  = {R1_k}k ohms\n"
        f"R2  = {R2_k}k ohms\n"
        f"V_CC = {V_CC} V"
    )
    ax2.text(0.1, 0.95, info_text, transform=ax2.transAxes, fontsize=12,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='lightyellow', edgecolor='gray'))
    
    plt.tight_layout()
    plt.show()

widgets.interact(
    interactive_load_line,
    R_C=widgets.IntSlider(min=470, max=4700, step=100, value=2200, description='R_C (ohms):'),
    R1_k=widgets.IntSlider(min=5, max=100, step=1, value=33, description='R1 (kohms):'),
    R2_k=widgets.IntSlider(min=2, max=47, step=1, value=10, description='R2 (kohms):'),
    R_E=widgets.IntSlider(min=0, max=1000, step=10, value=220, description='R_E (ohms):'),
    V_CC=widgets.FloatSlider(min=3, max=15, step=0.5, value=9.0, description='V_CC (V):')
);

---
## The Material Science Why — Saturation From the Silicon Up

When we drive a BJT into **saturation** (as a switch), something interesting happens at
the material level:

- In the **active region**, only the BE junction is forward-biased. The BC junction is
  reverse-biased, and its depletion region sweeps carriers into the collector.

- In **saturation**, we push so much base current that the collector voltage drops close
  to the emitter voltage. Now the BC junction also becomes **forward-biased**.

- With **both junctions forward-biased**, carriers are being injected from both sides into
  the base. The base region becomes flooded with excess minority carriers — this is called
  **charge storage**.

- **Consequence for switching**: When you try to turn the transistor OFF, you first have to
  remove all that stored charge from the base. This takes time — it is the **storage delay
  time (t_s)** listed on the datasheet. It is why BJTs are slower switches than MOSFETs
  for high-frequency applications.

This is also why driving a BJT deeper into saturation (more base current) makes it turn
off SLOWER — more stored charge to remove. There is a practical tradeoff between ensuring
reliable saturation and switching speed.

---
## Experiment 1 — 2N3904 as a Switch (LED Driver)

### Circuit

```
    V_CC (+5V bench supply)
         |
        [R_C = 150 ohm]
         |
        (A)---- LED (red, ~2V) ----+
                                   |
                                   C
  V_in (bench supply) ---[R_B = 1k]--- B --| 2N3904
                                   E
                                   |
                                  GND
```

### Procedure

1. **Build the circuit** on a breadboard
2. Set V_CC to 5.0V on the bench supply (use the main output)
3. Use a second supply output (or a resistor divider) for V_in

### Measurements with Klein MM300

**Step A: BJT OFF (V_in = 0V)**
- Measure V_CE (collector to emitter): Expected ~5V (full supply)
- Measure V_BE (base to emitter): Expected ~0V
- LED should be OFF

**Step B: BJT ON (V_in = 5V)**
- Measure V_CE(sat): Expected 0.1-0.3V
- Measure V_BE: Expected ~0.65-0.75V
- LED should be ON brightly

**Step C: Measure Beta**
- Measure voltage across R_C (or use Ohm's law with measured V_CE and supply voltage)
- I_C = (V_CC - V_LED - V_CE(sat)) / R_C
- I_B = (V_in - V_BE) / R_B
- With R_B = 1k: I_B is large, so forced beta ~ I_C / I_B should be low (switch is heavily saturated)
- Try R_B = 100k for a more accurate beta measurement in active region

Record your measurements in the table below:

In [ ]:
# Fill in your measurements here!

# BJT OFF (V_in = 0V)
V_CE_off_measured = None   # Your measurement in volts
V_BE_off_measured = None   # Your measurement in volts

# BJT ON (V_in = 5V, R_B = 1k)
V_CE_sat_measured = None   # Your measurement in volts
V_BE_on_measured = None    # Your measurement in volts

# Beta measurement (R_B = 100k)
V_RC_measured = None       # Voltage across R_C in volts
V_BE_beta_measured = None  # V_BE during beta measurement

# Calculate beta from your measurements
R_C_used = 150  # ohms
R_B_used = 100000  # ohms (for beta measurement)
V_supply = 5.0

if V_RC_measured is not None and V_BE_beta_measured is not None:
    I_C_measured = V_RC_measured / R_C_used
    I_B_measured = (V_supply - V_BE_beta_measured) / R_B_used
    beta_measured = I_C_measured / I_B_measured
    print(f"Your measured values:")
    print(f"  I_C = {I_C_measured*1e3:.2f} mA")
    print(f"  I_B = {I_B_measured*1e6:.1f} uA")
    print(f"  Beta (hFE) = {beta_measured:.0f}")
    print(f"  (Datasheet spec: 100-300 @ I_C=10mA, V_CE=1V)")
else:
    print("Fill in your measurements above and re-run this cell!")
    print("\nExpected values (approximate):")
    print(f"  V_CE(sat) ~ 0.1-0.3 V")
    print(f"  V_BE(on)  ~ 0.65-0.75 V")
    print(f"  Beta      ~ 100-300")

---
## Experiment 2 — Common-Emitter Amplifier

### Circuit

```
    V_CC (+9V bench supply)
      |           |
     [R1=33k]    [R_C=2.2k]
      |           |   
      +---B   C---+---[10uF cap]----> V_out (to oscilloscope CH1)
      |   | 2N3904
     [R2=10k] E
      |       |
      |      [R_E=220]
      |       |
     GND     GND
      
    V_in: Fnirsi signal generator -> [10uF cap] -> Base junction (at R1/R2 node)
    Signal: 1kHz sine wave, 50mV peak-to-peak
```

### Procedure

1. **Build the circuit** on a breadboard
2. Set bench supply to 9.0V for V_CC
3. **First, measure DC bias** (no signal):
   - V_B (base to ground): expected ~2.1V
   - V_E (emitter to ground): expected ~1.4V (V_B - 0.7)
   - V_C (collector to ground): expected ~4-5V (varies)
4. **Connect the signal generator** (Fnirsi 2C53T):
   - Set to sine wave, 1 kHz, 50 mV p-p
   - Connect through 10uF coupling capacitor to base node
5. **Observe on oscilloscope**:
   - CH1: Output at collector (through coupling cap)
   - CH2: Input signal
   - Measure the voltage gain: A_v = V_out(p-p) / V_in(p-p)
   - Observe the 180-degree phase inversion

### Expected Results
- Gain without R_E bypass: A_v ~ -R_C / R_E = -2200/220 = **-10** (about 10x amplification)
- 50mV input should produce ~500mV output (inverted)
- If you add a 100uF capacitor across R_E: gain jumps dramatically!

---
## Experiment 3 — 2N3906 PNP High-Side Switch

PNP transistors are used for **high-side switching** — controlling power from the positive rail.

```
    V_CC (+5V)
      |
      E
      | 2N3906 (PNP)
      B---[R_B = 1k]--- V_control
      C
      |
     (A)---- LED ----[R = 150 ohm]---- GND
```

### How PNP Switching Works

- **LED ON**: V_control = 0V (GND). This pulls the base 5V below the emitter, 
  forward-biasing the EB junction. Current flows E -> C -> LED.
- **LED OFF**: V_control = 5V (V_CC). Base and emitter are at the same voltage.
  No base current, BJT is in cutoff.

Note the logic is **inverted** compared to NPN: low input = ON, high input = OFF.

### Measurements
- V_EB (emitter to base) when ON: expected ~0.65-0.75V
- V_EC (emitter to collector) when ON: expected ~0.1-0.3V (saturation)
- Compare LED brightness to the NPN circuit — should be similar

---
## Simulation

**BJT Switch Circuit:**
[Falstad: NPN LED Switch](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKACURcAWEPKlbnyEooEKPE4gAZgEMANgBcW4yYuXLBKtRAhh+IPglkJF-GCnAxZ02VBlz5YFZr3g9SKPoJkJl3X76UHt6eAJI+kkA)

**Common-Emitter Amplifier:**
[Falstad: CE Amplifier](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKAHcRcAWTvPKFH34h+UEaImSBYOQlkgAbkJAsiKbHgIE+w0eMljJYbXt5JlBxCe5n+IcxavdTtw4gJHE0vj-0QNMkA)

---
## Datasheet Connection

### For the 2N3904 (NPN) — Switch Design

| Parameter | Datasheet Section | Value | Why It Matters |
|-----------|------------------|-------|----------------|
| V_CE(sat) | Electrical Characteristics | 0.2V @ I_C=10mA, I_B=1mA | Power wasted when ON |
| h_FE (min) | Electrical Characteristics | 100 (min) | Must use WORST case for switch design |
| I_C (max) | Absolute Maximum | 200 mA | Never exceed this! |
| P_D (max) | Absolute Maximum | 625 mW | V_CE * I_C must stay below this |
| t_on, t_off | Switching Characteristics | ~35ns, ~200ns | Limits switching speed |

### For the 2N3906 (PNP) — High-Side Switch

The 2N3906 is the PNP complement to the 2N3904. Nearly identical specs but with
reversed polarities.

### Key Datasheet Insight: Saturation Conditions

The datasheet specifies V_CE(sat) at a particular **I_C and I_B ratio**. For 2N3904:
- V_CE(sat) = 0.2V max at I_C = 10mA, I_B = 1mA (forced beta = 10)
- V_CE(sat) = 0.3V max at I_C = 50mA, I_B = 5mA

This confirms our design rule: use a forced beta of ~10 for reliable saturation.

In [ ]:
# Power dissipation analysis for BJT switch

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

I_C_range = np.linspace(0.001, 0.200, 200)  # 1mA to 200mA

# Left: Power in each state
V_CE_sat = 0.2
V_CC = 5.0

P_on = V_CE_sat * I_C_range  # Power when ON (saturated)
P_max_line = np.full_like(I_C_range, 0.625)  # 625 mW max

ax1.plot(I_C_range * 1e3, P_on * 1e3, 'r-', linewidth=2, label='P (ON, saturated)')
ax1.axhline(y=625, color='gray', linestyle='--', linewidth=2, label='P_D(max) = 625 mW')
ax1.fill_between(I_C_range * 1e3, P_on * 1e3, 625, alpha=0.1, color='green', label='Safe zone')

ax1.set_xlabel('I_C (mA)', fontsize=12)
ax1.set_ylabel('Power Dissipation (mW)', fontsize=12)
ax1.set_title('Power Dissipation in BJT Switch\n(Saturated, V_CE(sat) = 0.2V)', 
              fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.set_xlim(0, 200)
ax1.set_ylim(0, 700)

# Right: Danger zone — power during switching transition
# During transition, V_CE can be high AND I_C can be high simultaneously
t = np.linspace(0, 1, 1000)  # normalized time
# Simple model: V_CE falls while I_C rises
I_C_trans = 0.020 * (1 - np.exp(-10 * t))  # 20mA target
V_CE_trans = V_CC * np.exp(-8 * t) + V_CE_sat * (1 - np.exp(-8 * t))
P_trans = V_CE_trans * I_C_trans

ax2.plot(t * 100, V_CE_trans, 'b-', linewidth=2, label='V_CE (V)')
ax2.plot(t * 100, I_C_trans * 1e3, 'r-', linewidth=2, label='I_C (mA) / scale')
ax2.fill_between(t * 100, P_trans * 1e3, alpha=0.3, color='orange', label='P_diss (mW)')
ax2.plot(t * 100, P_trans * 1e3, 'orange', linewidth=2)

ax2.set_xlabel('Time (% of transition)', fontsize=12)
ax2.set_ylabel('Value', fontsize=12)
ax2.set_title('Power Spike During Switching Transition\n(This is where the heat happens!)', 
              fontsize=13, fontweight='bold')
ax2.legend(fontsize=10)

plt.tight_layout()
plt.show()

print("Key insight: A BJT switch wastes very little power when fully ON or fully OFF.")
print("The peak power dissipation happens DURING the transition between states.")
print("This is why fast switching is important for efficiency.")

---
## Checkpoint Questions

**Q1.** You want to use a 2N3904 to switch an LED that needs 20mA. Your input signal is 3.3V
(from a microcontroller). V_CC = 5V. The LED has a 2V forward drop. What values of R_C
and R_B do you need?

<details><summary>Answer</summary>

R_C = (V_CC - V_LED - V_CE(sat)) / I_LED = (5 - 2 - 0.2) / 0.020 = 140 ohms. Use 150 ohms.

I_B(design) = I_C / forced_beta = 20mA / 10 = 2 mA

R_B = (V_in - V_BE) / I_B = (3.3 - 0.7) / 2mA = 1300 ohms. Use 1.2k or 1k ohms.
</details>

**Q2.** Why do we use a "forced beta" of 10 instead of the typical beta of 200 when designing
a switch circuit?

<details><summary>Answer</summary>

Beta varies widely from part to part (100-300 for 2N3904). Using a forced beta of 10
ensures the transistor is driven well into saturation regardless of the actual beta.
This guarantees reliable switching with the lowest V_CE(sat). The excess base current
is "wasted" but it is small compared to the collector current.
</details>

**Q3.** In a common-emitter amplifier with R_C = 2.2k and R_E = 220 ohms (no bypass cap),
what is the approximate voltage gain? If you add a bypass capacitor across R_E and
I_C = 6 mA, what does the gain become?

<details><summary>Answer</summary>

Without bypass: A_v ~ -R_C / R_E = -2200 / 220 = -10 (gain of 10, inverted)

With bypass: r_e = 26mV / 6mA = 4.3 ohms
A_v ~ -R_C / r_e = -2200 / 4.3 = -512 (gain of ~500, inverted)

This demonstrates why R_E stabilizes gain (at the cost of lower gain).
</details>

**Q4.** For the PNP high-side switch (2N3906), what voltage on the base turns the LED ON?
What turns it OFF? Why is this logic inverted compared to the NPN?

<details><summary>Answer</summary>

ON: V_base = 0V (ground). The emitter is at V_CC = 5V, so V_EB = 5V - 0V = 5V, which
forward-biases the EB junction (through R_B).

OFF: V_base = 5V (V_CC). V_EB = 5V - 5V = 0V, no forward bias.

The logic is inverted because the PNP emitter connects to V_CC (the positive rail) and
the base must be pulled LOW relative to the emitter to turn on.
</details>

**Q5.** You measure V_CE = 2.5V across your 2N3904 switch when it should be saturated.
What is likely wrong?

<details><summary>Answer</summary>

The BJT is NOT saturated — it is in the active region. Likely causes:
- R_B is too large (not enough base current)
- The input voltage V_in is too low
- The base resistor is not connected properly
- The collector load is drawing more current than expected

Solution: Decrease R_B to increase base current, or verify all connections.
</details>